# Package Installation Section #

In [2]:
'''
pip install yfinance  # Install the yfinance library
pip install openpyxl
pip install pandas
pip install re
'''

'\npip install yfinance  # Install the yfinance library\npip install openpyxl\npip install pandas\npip install re\n'

# Package Import Section #

In [38]:
# import yfinance as yf
import pandas as pd
from openpyxl import load_workbook
import re
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# Download S&P500 Return Index and Volatility Index (VIX) from Yahoo Finance #

In [5]:
# Define the tickers for S&P 500 (^GSPC) and VIX (^VIX)
tickers = ["^GSPC", "^VIX"]

# Download weekly data from Yahoo Finance
data = yf.download(tickers, start="2014-12-01", end="2024-11-30", interval="1wk", group_by="ticker")

# Extract closing prices
sp500_close = data["^GSPC"]["Close"]
vix_close = data["^VIX"]["Close"]

# Calculate weekly returns for S&P 500 (percentage change)
sp500_return = sp500_close.pct_change() * 100

# Combine data into a single DataFrame
sp500_vix = pd.DataFrame({
    "Date": sp500_close.index,
    "SP500_Return (%)": sp500_return,
    "VIX_Close": vix_close
}).set_index("Date")

# Drop rows with NaN values (e.g., first row for returns calculation)
sp500_vix.dropna(inplace=True)

# Save to a CSV file
#combined_data.to_csv("weekly_sp500_vix.csv")

sp500_vix

[*********************100%***********************]  2 of 2 completed


,SP500_Return (%),VIX_Close
Date,,
2014-12-08,-3.519380,21.080000
2014-12-15,3.412022,16.490000
2014-12-22,0.875093,14.500000
2014-12-29,-1.463544,17.790001
2015-01-05,-0.650563,17.549999
...,...,...
2024-10-28,-1.365680,21.879999
2024-11-04,4.656128,14.940000
2024-11-11,-2.083547,16.139999


# Import Downloaded Dataset from Refinitiv (weekly from 01/12/2014 to 30/11/2024) #

In [2]:
# Load the workbook
file_path = "Request finish.xlsm"
load_file = load_workbook(file_path, keep_vba=True)

# Access the worksheet
stock_data = load_file['full']

# Extract data from the worksheet into a pandas DataFrame
data = stock_data.values  # Extract the data as a generator of rows
columns = next(data)  # Get the first row as column names
df_stock = pd.DataFrame(data, columns=columns)  # Create DataFrame

# Display the DataFrame
df_stock.head()

,Name,Code,2014-01-10 00:00:00,2014-01-17 00:00:00,2014-01-24 00:00:00,2014-01-31 00:00:00,2014-02-07 00:00:00,2014-02-14 00:00:00,2014-02-21 00:00:00,2014-02-28 00:00:00,...,2024-09-27 00:00:00,2024-10-04 00:00:00,2024-10-11 00:00:00,2024-10-18 00:00:00,2024-10-25 00:00:00,2024-11-01 00:00:00,2024-11-08 00:00:00,2024-11-15 00:00:00,2024-11-22 00:00:00,2024-11-29 00:00:00
0,PALANTIR TECHNOLOGIES A,US69608A1088(P),None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,36.84,40.01,43.51,42.97,44.860,41.92,58.39,65.77,64.35,67.08
1,PALANTIR TECHNOLOGIES A - ASK PRICE,US69608A1088(PA),None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,36.86,40.01,43.50,42.96,44.860,41.94,58.44,65.79,64.33,67.09
2,PALANTIR TECHNOLOGIES A - BID PRICE,US69608A1088(PB),None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,36.85,40.00,43.49,42.95,44.850,41.93,58.43,65.78,64.32,67.06
3,PALANTIR TECHNOLOGIES A - PRICE HIGH,US69608A1088(PH),None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.47,40.29,44.38,42.99,45.070,42.57,58.48,66.00,64.44,67.16
4,PALANTIR TECHNOLOGIES A - PRICE LOW,US69608A1088(PL),None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,36.59,39.40,42.62,41.65,43.645,41.59,55.30,60.91,61.37,65.47


In [3]:
# Make a copy of df_stock
df = df_stock.copy()

# Transpose the Dataset to Long Format #

In [4]:
# Step 1: delete data before December 2024 to align with the index data

# Define the date range
start_date = "2014-01-10 00:00:00"
end_date = "2014-11-28 00:00:00"

# Separate datetime columns
datetime_cols = pd.to_datetime(df.columns[2:], format="%Y-%m-%d %H:%M:%S", errors='coerce')

# Filter columns outside the specified date range
columns_to_keep = ~datetime_cols.to_series().between(start_date, end_date)

# Keep only the non-datetime columns and filtered datetime columns
df = df.iloc[:, :2].join(df.iloc[:, 2:].loc[:, columns_to_keep])

# Reconstruct the column names: combine non-datetime and filtered datetime columns
#df.columns = list(df.columns[:2]) + list(datetime_cols[columns_to_keep])

# Display the resulting DataFrame
df.head()

,Name,Code,2014-12-05 00:00:00,2014-12-12 00:00:00,2014-12-19 00:00:00,2014-12-26 00:00:00,2015-01-02 00:00:00,2015-01-09 00:00:00,2015-01-16 00:00:00,2015-01-23 00:00:00,...,2024-09-27 00:00:00,2024-10-04 00:00:00,2024-10-11 00:00:00,2024-10-18 00:00:00,2024-10-25 00:00:00,2024-11-01 00:00:00,2024-11-08 00:00:00,2024-11-15 00:00:00,2024-11-22 00:00:00,2024-11-29 00:00:00
0,PALANTIR TECHNOLOGIES A,US69608A1088(P),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,36.84,40.01,43.51,42.97,44.860,41.92,58.39,65.77,64.35,67.08
1,PALANTIR TECHNOLOGIES A - ASK PRICE,US69608A1088(PA),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,36.86,40.01,43.50,42.96,44.860,41.94,58.44,65.79,64.33,67.09
2,PALANTIR TECHNOLOGIES A - BID PRICE,US69608A1088(PB),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,36.85,40.00,43.49,42.95,44.850,41.93,58.43,65.78,64.32,67.06
3,PALANTIR TECHNOLOGIES A - PRICE HIGH,US69608A1088(PH),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.47,40.29,44.38,42.99,45.070,42.57,58.48,66.00,64.44,67.16
4,PALANTIR TECHNOLOGIES A - PRICE LOW,US69608A1088(PL),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,36.59,39.40,42.62,41.65,43.645,41.59,55.30,60.91,61.37,65.47


In [5]:
# Step 2: create a new column with only company names and a new column name with only variable names

# Create a new column 'company_name' and set it to None
df['company_name'] = None

# Iterate every 35 rows to extract the company name and forward-fill
for start in range(0, len(df), 35):
    # Extract the company name at the start of each 35-row block
    company_name = df.loc[start, 'Name']

    # Assign this company name to all rows in the current block
    df.loc[start:start+34, 'company_name'] = company_name

# Forward-fill the remaining rows
df['company_name'] = df['company_name'].ffill()

# Create a new column 'var_name' and set it to None
df['var_name'] = None

# Create a list of 35 different variables
variables = ["close_price", "ask_price", "bid_price", "price_high", "price_low", "price_open",
             "turnover_value", "turnover_volume", "number_trades", "vwap", "trading_volume_wa",
             "total_return", "pe", "ptbv", "dividend_yield", "operating_pm", "gross_pm",
             "net_operating_income", "net_income", "roe", "roic", "roa", "ebit", "ebitda", "dpps",
             "current_ratio", "quick_ratio", "inventory_turnover", "asset_turnover",
             "tdce", "ltdce", "interest_cover1", "interest_cover2", "cash_dividend", "shares_outstanding"]

# Iterate over every 35 rows to assign variable names
for start in range(0, len(df), 35):
    # Assign variable names to the 'var_name' column for the current block
    var_names_to_assign = variables[:len(df.loc[start:start+34])]
    df.loc[start:start+34, 'var_name'] = var_names_to_assign

# Move 'company_name' and 'var_name' to the correct positions
df = df[['company_name', 'var_name'] + [col for col in df.columns if col not in ['company_name', 'var_name']]]

# View the resulting DataFrame
df

,company_name,var_name,Name,Code,2014-12-05 00:00:00,2014-12-12 00:00:00,2014-12-19 00:00:00,2014-12-26 00:00:00,2015-01-02 00:00:00,2015-01-09 00:00:00,...,2024-09-27 00:00:00,2024-10-04 00:00:00,2024-10-11 00:00:00,2024-10-18 00:00:00,2024-10-25 00:00:00,2024-11-01 00:00:00,2024-11-08 00:00:00,2024-11-15 00:00:00,2024-11-22 00:00:00,2024-11-29 00:00:00
0,PALANTIR TECHNOLOGIES A,close_price,PALANTIR TECHNOLOGIES A,US69608A1088(P),NaN,NaN,NaN,NaN,NaN,NaN,...,36.84,40.01,43.51,42.97,44.860,41.92,58.39,65.77,64.35,67.08
1,PALANTIR TECHNOLOGIES A,ask_price,PALANTIR TECHNOLOGIES A - ASK PRICE,US69608A1088(PA),NaN,NaN,NaN,NaN,NaN,NaN,...,36.86,40.01,43.50,42.96,44.860,41.94,58.44,65.79,64.33,67.09
2,PALANTIR TECHNOLOGIES A,bid_price,PALANTIR TECHNOLOGIES A - BID PRICE,US69608A1088(PB),NaN,NaN,NaN,NaN,NaN,NaN,...,36.85,40.00,43.49,42.95,44.850,41.93,58.43,65.78,64.32,67.06
3,PALANTIR TECHNOLOGIES A,price_high,PALANTIR TECHNOLOGIES A - PRICE HIGH,US69608A1088(PH),NaN,NaN,NaN,NaN,NaN,NaN,...,37.47,40.29,44.38,42.99,45.070,42.57,58.48,66.00,64.44,67.16
4,PALANTIR TECHNOLOGIES A,price_low,PALANTIR TECHNOLOGIES A - PRICE LOW,US69608A1088(PL),NaN,NaN,NaN,NaN,NaN,NaN,...,36.59,39.40,42.62,41.65,43.645,41.59,55.30,60.91,61.37,65.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,MASTERCARD,ltdce,"MASTERCARD, INC. - LONG TERM DEBT % COMMON EQUITY",US57636Q1040(WC08226),22.090,22.090,22.090,22.090,54.610,54.610,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6996,MASTERCARD,interest_cover1,"MASTERCARD, INC. - EBIT/TOT INT EXPENSE RATIO",US57636Q1040(WC08291),106.813,106.813,106.813,106.813,82.279,82.279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6997,MASTERCARD,interest_cover2,"MASTERCARD, INC. - INTEREST COVER",US57636Q1040(ICBT),106.810,106.810,106.810,106.810,82.280,82.280,...,24.72,24.72,24.72,24.72,24.720,24.72,24.72,24.72,24.72,24.72
6998,MASTERCARD,cash_dividend,"MASTERCARD, INC. - CASH DIVIDENDS PAID - TOTAL",US57636Q1040(WC04551),515000.000,515000.000,515000.000,515000.000,727000.000,727000.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Step 3: identify and count #ERROR occurrences:

# Filter rows where the 'Name' column contains '#ERROR'
error_rows = df[df['Name'] == '#ERROR']

# Group by 'var_name' and count occurrences of '#ERROR'
error_counts = error_rows.groupby('var_name').size().reset_index(name='error_count')

# View the result
error_counts

,var_name,error_count
0,current_ratio,43
1,dpps,2
2,gross_pm,30
3,interest_cover1,4
4,inventory_turnover,76
5,ltdce,3
6,net_operating_income,200
7,ptbv,2
8,quick_ratio,43
9,roe,3


In [7]:
# Step 4: drop variables with error_count > 60 (30%) for all companies

# Identify variables with error_count > 60
variables_to_drop = error_counts[error_counts['error_count'] > 60]['var_name']

# Drop these variables from the dataset
df = df[~df['var_name'].isin(variables_to_drop)]

# View the updated DataFrame
df                         # Two variables(inventory_turnover and net_operating_income) are dropped at this stage

,company_name,var_name,Name,Code,2014-12-05 00:00:00,2014-12-12 00:00:00,2014-12-19 00:00:00,2014-12-26 00:00:00,2015-01-02 00:00:00,2015-01-09 00:00:00,...,2024-09-27 00:00:00,2024-10-04 00:00:00,2024-10-11 00:00:00,2024-10-18 00:00:00,2024-10-25 00:00:00,2024-11-01 00:00:00,2024-11-08 00:00:00,2024-11-15 00:00:00,2024-11-22 00:00:00,2024-11-29 00:00:00
0,PALANTIR TECHNOLOGIES A,close_price,PALANTIR TECHNOLOGIES A,US69608A1088(P),NaN,NaN,NaN,NaN,NaN,NaN,...,36.84,40.01,43.51,42.97,44.860,41.92,58.39,65.77,64.35,67.08
1,PALANTIR TECHNOLOGIES A,ask_price,PALANTIR TECHNOLOGIES A - ASK PRICE,US69608A1088(PA),NaN,NaN,NaN,NaN,NaN,NaN,...,36.86,40.01,43.50,42.96,44.860,41.94,58.44,65.79,64.33,67.09
2,PALANTIR TECHNOLOGIES A,bid_price,PALANTIR TECHNOLOGIES A - BID PRICE,US69608A1088(PB),NaN,NaN,NaN,NaN,NaN,NaN,...,36.85,40.00,43.49,42.95,44.850,41.93,58.43,65.78,64.32,67.06
3,PALANTIR TECHNOLOGIES A,price_high,PALANTIR TECHNOLOGIES A - PRICE HIGH,US69608A1088(PH),NaN,NaN,NaN,NaN,NaN,NaN,...,37.47,40.29,44.38,42.99,45.070,42.57,58.48,66.00,64.44,67.16
4,PALANTIR TECHNOLOGIES A,price_low,PALANTIR TECHNOLOGIES A - PRICE LOW,US69608A1088(PL),NaN,NaN,NaN,NaN,NaN,NaN,...,36.59,39.40,42.62,41.65,43.645,41.59,55.30,60.91,61.37,65.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,MASTERCARD,ltdce,"MASTERCARD, INC. - LONG TERM DEBT % COMMON EQUITY",US57636Q1040(WC08226),22.090,22.090,22.090,22.090,54.610,54.610,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6996,MASTERCARD,interest_cover1,"MASTERCARD, INC. - EBIT/TOT INT EXPENSE RATIO",US57636Q1040(WC08291),106.813,106.813,106.813,106.813,82.279,82.279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6997,MASTERCARD,interest_cover2,"MASTERCARD, INC. - INTEREST COVER",US57636Q1040(ICBT),106.810,106.810,106.810,106.810,82.280,82.280,...,24.72,24.72,24.72,24.72,24.720,24.72,24.72,24.72,24.72,24.72
6998,MASTERCARD,cash_dividend,"MASTERCARD, INC. - CASH DIVIDENDS PAID - TOTAL",US57636Q1040(WC04551),515000.000,515000.000,515000.000,515000.000,727000.000,727000.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Step 5: clean the company code column and drop the "Name" and "Code" column

# Reset index to ensure integer-based indexing
df = df.reset_index(drop=True)

# Initialize the 'company_code' column
df['company_code'] = None  # Avoid SettingWithCopyWarning as this directly modifies the DataFrame

# Iterate through the DataFrame in blocks of 33 rows
for start in range(0, len(df), 33):
    # Extract the first value in the current block
    first_value = df.iloc[start, df.columns.get_loc('Code')]  # Use .iloc for integer-based indexing

    # Initialize company_code
    company_code = None

    # Check if first_value is a string and matches the desired pattern
    if isinstance(first_value, str):
        match = re.match(r'([^\(]+)', first_value)  # Match company codes starting with "US"
        if match:
            company_code = match.group(1)

    # Fill the current block (33 rows) with the extracted company code
    df.loc[start:start+32, 'company_code'] = company_code

# Move the 'company_code' column to the second position
columns = list(df.columns)  # Get the list of columns
columns.remove('company_code')  # Remove 'company_code_filled' from the list
columns.insert(1, 'company_code')  # Insert it at the second position
df = df[columns]  # Reorder the DataFrame

# View the result
df

,company_name,company_code,var_name,Name,Code,2014-12-05 00:00:00,2014-12-12 00:00:00,2014-12-19 00:00:00,2014-12-26 00:00:00,2015-01-02 00:00:00,...,2024-09-27 00:00:00,2024-10-04 00:00:00,2024-10-11 00:00:00,2024-10-18 00:00:00,2024-10-25 00:00:00,2024-11-01 00:00:00,2024-11-08 00:00:00,2024-11-15 00:00:00,2024-11-22 00:00:00,2024-11-29 00:00:00
0,PALANTIR TECHNOLOGIES A,US69608A1088,close_price,PALANTIR TECHNOLOGIES A,US69608A1088(P),NaN,NaN,NaN,NaN,NaN,...,36.84,40.01,43.51,42.97,44.860,41.92,58.39,65.77,64.35,67.08
1,PALANTIR TECHNOLOGIES A,US69608A1088,ask_price,PALANTIR TECHNOLOGIES A - ASK PRICE,US69608A1088(PA),NaN,NaN,NaN,NaN,NaN,...,36.86,40.01,43.50,42.96,44.860,41.94,58.44,65.79,64.33,67.09
2,PALANTIR TECHNOLOGIES A,US69608A1088,bid_price,PALANTIR TECHNOLOGIES A - BID PRICE,US69608A1088(PB),NaN,NaN,NaN,NaN,NaN,...,36.85,40.00,43.49,42.95,44.850,41.93,58.43,65.78,64.32,67.06
3,PALANTIR TECHNOLOGIES A,US69608A1088,price_high,PALANTIR TECHNOLOGIES A - PRICE HIGH,US69608A1088(PH),NaN,NaN,NaN,NaN,NaN,...,37.47,40.29,44.38,42.99,45.070,42.57,58.48,66.00,64.44,67.16
4,PALANTIR TECHNOLOGIES A,US69608A1088,price_low,PALANTIR TECHNOLOGIES A - PRICE LOW,US69608A1088(PL),NaN,NaN,NaN,NaN,NaN,...,36.59,39.40,42.62,41.65,43.645,41.59,55.30,60.91,61.37,65.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6595,MASTERCARD,US57636Q1040,ltdce,"MASTERCARD, INC. - LONG TERM DEBT % COMMON EQUITY",US57636Q1040(WC08226),22.090,22.090,22.090,22.090,54.610,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6596,MASTERCARD,US57636Q1040,interest_cover1,"MASTERCARD, INC. - EBIT/TOT INT EXPENSE RATIO",US57636Q1040(WC08291),106.813,106.813,106.813,106.813,82.279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6597,MASTERCARD,US57636Q1040,interest_cover2,"MASTERCARD, INC. - INTEREST COVER",US57636Q1040(ICBT),106.810,106.810,106.810,106.810,82.280,...,24.72,24.72,24.72,24.72,24.720,24.72,24.72,24.72,24.72,24.72
6598,MASTERCARD,US57636Q1040,cash_dividend,"MASTERCARD, INC. - CASH DIVIDENDS PAID - TOTAL",US57636Q1040(WC04551),515000.000,515000.000,515000.000,515000.000,727000.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Drop the "Name" and "Code" columns
df = df.drop(columns=['Name', 'Code'])

# View the result
df

,company_name,company_code,var_name,2014-12-05 00:00:00,2014-12-12 00:00:00,2014-12-19 00:00:00,2014-12-26 00:00:00,2015-01-02 00:00:00,2015-01-09 00:00:00,2015-01-16 00:00:00,...,2024-09-27 00:00:00,2024-10-04 00:00:00,2024-10-11 00:00:00,2024-10-18 00:00:00,2024-10-25 00:00:00,2024-11-01 00:00:00,2024-11-08 00:00:00,2024-11-15 00:00:00,2024-11-22 00:00:00,2024-11-29 00:00:00
0,PALANTIR TECHNOLOGIES A,US69608A1088,close_price,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,36.84,40.01,43.51,42.97,44.860,41.92,58.39,65.77,64.35,67.08
1,PALANTIR TECHNOLOGIES A,US69608A1088,ask_price,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,36.86,40.01,43.50,42.96,44.860,41.94,58.44,65.79,64.33,67.09
2,PALANTIR TECHNOLOGIES A,US69608A1088,bid_price,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,36.85,40.00,43.49,42.95,44.850,41.93,58.43,65.78,64.32,67.06
3,PALANTIR TECHNOLOGIES A,US69608A1088,price_high,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.47,40.29,44.38,42.99,45.070,42.57,58.48,66.00,64.44,67.16
4,PALANTIR TECHNOLOGIES A,US69608A1088,price_low,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,36.59,39.40,42.62,41.65,43.645,41.59,55.30,60.91,61.37,65.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6595,MASTERCARD,US57636Q1040,ltdce,22.090,22.090,22.090,22.090,54.610,54.610,54.610,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6596,MASTERCARD,US57636Q1040,interest_cover1,106.813,106.813,106.813,106.813,82.279,82.279,82.279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6597,MASTERCARD,US57636Q1040,interest_cover2,106.810,106.810,106.810,106.810,82.280,82.280,82.280,...,24.72,24.72,24.72,24.72,24.720,24.72,24.72,24.72,24.72,24.72
6598,MASTERCARD,US57636Q1040,cash_dividend,515000.000,515000.000,515000.000,515000.000,727000.000,727000.000,727000.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Make a copy of df
df_short = df.copy()

df_short['company_name'].nunique()   # Check the number of unique company names

200

In [11]:
df_short['company_code'].nunique()   # Check the number of unique company codes

200

In [12]:
# Step 6: transpose the dataset from short form to long form

# Identify date columns
date_columns = df_short.columns[3:]

# Transform from Short Form to Long Form using melt()
df_tran = df_short.melt(
    id_vars=['company_name', 'company_code', 'var_name'],  # Columns to keep
    value_vars=date_columns,  # Date columns to "unpivot"
    var_name='date',  # New column for date names
    value_name='value'  # Column for values
)

df_tran

,company_name,company_code,var_name,date,value
0,PALANTIR TECHNOLOGIES A,US69608A1088,close_price,2014-12-05,NaN
1,PALANTIR TECHNOLOGIES A,US69608A1088,ask_price,2014-12-05,NaN
2,PALANTIR TECHNOLOGIES A,US69608A1088,bid_price,2014-12-05,NaN
3,PALANTIR TECHNOLOGIES A,US69608A1088,price_high,2014-12-05,NaN
4,PALANTIR TECHNOLOGIES A,US69608A1088,price_low,2014-12-05,NaN
...,...,...,...,...,...
3445195,MASTERCARD,US57636Q1040,ltdce,2024-11-29,NaN
3445196,MASTERCARD,US57636Q1040,interest_cover1,2024-11-29,NaN
3445197,MASTERCARD,US57636Q1040,interest_cover2,2024-11-29,24.72
3445198,MASTERCARD,US57636Q1040,cash_dividend,2024-11-29,NaN


In [13]:
# Ensure 'date' column is in datetime format for consistency
df_tran['date'] = pd.to_datetime(df_tran['date'])

# Reshape: Pivot to separate variables (var1, var2, ..., var33) into columns
df_long = df_tran.pivot_table(
    index=['company_name', 'company_code', 'date'],  # Group by company and date
    columns='var_name',  # Pivot on variable names
    values='value'  # Values column
).reset_index()

# Clean column names
df_long.columns.name = None  # Remove column grouping name

# The resulting `df_long` is in long form with desired structure.
df_long

,company_name,company_code,date,ask_price,asset_turnover,bid_price,cash_dividend,close_price,current_ratio,dividend_yield,...,roa,roe,roic,shares_outstanding,tdce,total_return,trading_volume_wa,turnover_value,turnover_volume,vwap
0,3M,US88579Y1010,2014-12-05,135.6365,1.05,135.6114,2216000.0,135.6699,1.96,2.11,...,15.80,32.38,22.79,635135.0,52.61,5622.58,1782344.0,NaN,14274.8,135.7130
1,3M,US88579Y1010,2014-12-12,131.4896,1.05,131.4812,2216000.0,131.3642,1.96,2.18,...,15.80,32.38,22.79,635135.0,52.61,5444.14,1782344.0,NaN,14913.8,132.0684
2,3M,US88579Y1010,2014-12-19,138.3705,1.05,138.3621,2216000.0,138.3537,1.96,2.48,...,15.80,32.38,22.79,635135.0,52.61,5733.81,1782344.0,1006228.0,21761.2,138.1173
3,3M,US88579Y1010,2014-12-26,139.0477,1.05,139.0226,2216000.0,139.0059,1.96,2.47,...,15.80,32.38,22.79,635135.0,52.61,5760.84,1782344.0,394893.9,8217.5,139.2474
4,3M,US88579Y1010,2015-01-02,137.1414,0.94,137.1163,2561000.0,137.1665,1.54,2.50,...,15.77,38.95,23.05,609330.0,92.61,5684.61,1821209.0,349564.0,7354.4,136.9276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103205,ZEBRA TECHNOLOGIES 'A',US9892071054,2024-11-01,384.6499,NaN,384.4900,NaN,384.6399,NaN,0.00,...,NaN,NaN,NaN,NaN,NaN,9484.27,NaN,1593302.0,4194.4,384.1338
103206,ZEBRA TECHNOLOGIES 'A',US9892071054,2024-11-08,399.3000,NaN,399.0500,NaN,399.3101,NaN,0.00,...,NaN,NaN,NaN,NaN,NaN,9846.00,NaN,798172.1,2052.8,399.8765
103207,ZEBRA TECHNOLOGIES 'A',US9892071054,2024-11-15,385.8899,NaN,385.7100,NaN,385.9099,NaN,0.00,...,NaN,NaN,NaN,NaN,NaN,9515.59,NaN,688635.1,1732.4,386.1726
103208,ZEBRA TECHNOLOGIES 'A',US9892071054,2024-11-22,397.1899,NaN,396.9900,NaN,397.1899,NaN,0.00,...,NaN,NaN,NaN,NaN,NaN,9793.72,NaN,710837.3,1836.0,396.1980


In [14]:
# Check the count of observations for a company (e.g., "PALANTIR TECHNOLOGIES A")

company_name = 'PALANTIR TECHNOLOGIES A'  # Replace with the company name you're interested in

# Filter the dataset to select only rows for the given company
company_data = df_long[df_long['company_name'] == company_name]

# Get the count of observations (rows) for that company
company_observation_count = company_data.shape[0]

print(f"Number of observations for {company_name}: {company_observation_count}")

Number of observations for PALANTIR TECHNOLOGIES A: 361


In [15]:
df_long.groupby('company_name').size().sort_values(ascending=True)  # Observation number ranked in ascending order

company_name
GE VERNOVA                 205
VERALTO                    257
PALANTIR TECHNOLOGIES A    361
CONSTELLATION ENERGY       361
CARRIER GLOBAL             413
                          ... 
F5                         522
FAIR ISAAC                 522
FASTENAL                   522
JOHNSON CONTROLS INTL.     522
ZEBRA TECHNOLOGIES 'A'     522
Length: 200, dtype: int64

In [16]:
df_long.groupby('company_name').size().sum() # Check the total observation number for all companies

103210

# Handling Missing values #

In [17]:
# Make a copy of df_long and check for missing value
df_mis = df_long.copy()

print(df_mis.isna().sum())

company_name              0
company_code              0
date                      0
ask_price              5421
asset_turnover         9052
bid_price              5421
cash_dividend          8747
close_price            2029
current_ratio         29477
dividend_yield         2029
dpps                  22326
ebit                   9370
ebitda                10210
gross_pm              23219
interest_cover1       12714
interest_cover2         210
ltdce                 14061
net_income             8634
number_trades         88264
operating_pm           8634
pe                     9897
price_high             5316
price_low              5316
price_open             5316
ptbv                   2587
quick_ratio           29582
roa                    9848
roe                   14007
roic                   9739
shares_outstanding     9056
tdce                   9056
total_return           2029
trading_volume_wa     11765
turnover_value         2431
turnover_volume        2029
vwap                

In [18]:
# Calculate the missing value percentage for each variable
missing_percentage = df_mis.isna().mean()
# Sort the percentages in descending order and round to 2 decimal places
sorted_missing_percentage = missing_percentage.sort_values(ascending=False).round(2)

# Print the sorted percentages
print(sorted_missing_percentage)

number_trades         0.86
quick_ratio           0.29
current_ratio         0.29
gross_pm              0.22
dpps                  0.22
ltdce                 0.14
roe                   0.14
interest_cover1       0.12
trading_volume_wa     0.11
ebitda                0.10
pe                    0.10
roa                   0.10
roic                  0.09
ebit                  0.09
shares_outstanding    0.09
tdce                  0.09
asset_turnover        0.09
cash_dividend         0.08
net_income            0.08
operating_pm          0.08
ask_price             0.05
bid_price             0.05
vwap                  0.05
price_high            0.05
price_low             0.05
price_open            0.05
ptbv                  0.03
turnover_value        0.02
turnover_volume       0.02
dividend_yield        0.02
close_price           0.02
total_return          0.02
interest_cover2       0.00
date                  0.00
company_code          0.00
company_name          0.00
dtype: float64


In [19]:
# Set a threshold for dropping variables (e.g., 30% missing)
threshold = 0.3

# Drop variables with missing percentage above the threshold
columns_to_drop = missing_percentage[missing_percentage > threshold].index
df_mis = df_mis.drop(columns=columns_to_drop)

print(f"Dropped columns: {columns_to_drop.tolist()}")
print("Remaining columns after dropping:")
print(df_mis.columns)

Dropped columns: ['number_trades']
Remaining columns after dropping:
Index(['company_name', 'company_code', 'date', 'ask_price', 'asset_turnover',
       'bid_price', 'cash_dividend', 'close_price', 'current_ratio',
       'dividend_yield', 'dpps', 'ebit', 'ebitda', 'gross_pm',
       'interest_cover1', 'interest_cover2', 'ltdce', 'net_income',
       'operating_pm', 'pe', 'price_high', 'price_low', 'price_open', 'ptbv',
       'quick_ratio', 'roa', 'roe', 'roic', 'shares_outstanding', 'tdce',
       'total_return', 'trading_volume_wa', 'turnover_value',
       'turnover_volume', 'vwap'],
      dtype='object')


In [20]:
# Function to handle missing data
def handle_missing_data(df_mis):
    # List of variables (columns starting from the 4th column)
    variable_columns = df_mis.columns[3:]

    for var in variable_columns:
        for company in df_mis['company_name'].unique():
            # Filter data for the specific company
            company_mask = df_mis['company_name'] == company
            var_data = df_mis.loc[company_mask, var]

            # Case 1: Entire column for the variable is missing for this company
            if var_data.isna().all():
                # Calculate the mean of the variable across other companies
                mean_value = df_mis.loc[df_mis['company_name'] != company, var].mean()
                df_mis.loc[company_mask, var] = mean_value

            else:
                # Case 2: Missing data at the beginning
                if var_data.isna().iloc[0]:
                    df_mis.loc[company_mask, var] = var_data.bfill()

                # Case 3: Missing data at the end
                if var_data.isna().iloc[-1]:
                    df_mis.loc[company_mask, var] = var_data.ffill()

                # Case 4: Missing data in between (including jumps)
                if var_data.isna().sum() > 0:
                    # Interpolate and assign to the correct company and variable
                    interpolated_values = var_data.interpolate()
                    df_mis.loc[company_mask, var] = interpolated_values

    return df_mis

# Apply the function to your dataset
df_filled = handle_missing_data(df_mis)

# View the result
df_filled

,company_name,company_code,date,ask_price,asset_turnover,bid_price,cash_dividend,close_price,current_ratio,dividend_yield,...,roa,roe,roic,shares_outstanding,tdce,total_return,trading_volume_wa,turnover_value,turnover_volume,vwap
0,3M,US88579Y1010,2014-12-05,135.6365,1.05,135.6114,2216000.0,135.6699,1.96,2.11,...,15.80,32.38,22.79,635135.0,52.61,5622.58,1782344.0,NaN,14274.8,135.7130
1,3M,US88579Y1010,2014-12-12,131.4896,1.05,131.4812,2216000.0,131.3642,1.96,2.18,...,15.80,32.38,22.79,635135.0,52.61,5444.14,1782344.0,NaN,14913.8,132.0684
2,3M,US88579Y1010,2014-12-19,138.3705,1.05,138.3621,2216000.0,138.3537,1.96,2.48,...,15.80,32.38,22.79,635135.0,52.61,5733.81,1782344.0,1006228.0,21761.2,138.1173
3,3M,US88579Y1010,2014-12-26,139.0477,1.05,139.0226,2216000.0,139.0059,1.96,2.47,...,15.80,32.38,22.79,635135.0,52.61,5760.84,1782344.0,394893.9,8217.5,139.2474
4,3M,US88579Y1010,2015-01-02,137.1414,0.94,137.1163,2561000.0,137.1665,1.54,2.50,...,15.77,38.95,23.05,609330.0,92.61,5684.61,1821209.0,349564.0,7354.4,136.9276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103205,ZEBRA TECHNOLOGIES 'A',US9892071054,2024-11-01,384.6499,0.67,384.4900,0.0,384.6399,1.05,0.00,...,5.73,10.26,8.01,51379.0,73.12,9484.27,535725.0,1593302.0,4194.4,384.1338
103206,ZEBRA TECHNOLOGIES 'A',US9892071054,2024-11-08,399.3000,0.67,399.0500,0.0,399.3101,1.05,0.00,...,5.73,10.26,8.01,51379.0,73.12,9846.00,535725.0,798172.1,2052.8,399.8765
103207,ZEBRA TECHNOLOGIES 'A',US9892071054,2024-11-15,385.8899,0.67,385.7100,0.0,385.9099,1.05,0.00,...,5.73,10.26,8.01,51379.0,73.12,9515.59,535725.0,688635.1,1732.4,386.1726
103208,ZEBRA TECHNOLOGIES 'A',US9892071054,2024-11-22,397.1899,0.67,396.9900,0.0,397.1899,1.05,0.00,...,5.73,10.26,8.01,51379.0,73.12,9793.72,535725.0,710837.3,1836.0,396.1980


In [21]:
# Check for missing values
print(df_filled.isna().sum())

company_name             0
company_code             0
date                     0
ask_price             2065
asset_turnover         418
bid_price             2065
cash_dividend          113
close_price           2029
current_ratio          422
dividend_yield        2029
dpps                  3838
ebit                   160
ebitda                 478
gross_pm                 0
interest_cover1       1296
interest_cover2        210
ltdce                  955
net_income               0
operating_pm             0
pe                    4259
price_high            2031
price_low             2031
price_open            2031
ptbv                  1860
quick_ratio            527
roa                    901
roe                   1266
roic                   949
shares_outstanding     422
tdce                   422
total_return          2029
trading_volume_wa     1534
turnover_value        2431
turnover_volume       2029
vwap                  2031
dtype: int64


In [22]:
# Function to handle remaining missing data
def handle_remaining_missing_data(df_mis):
    '''
    # Step 1: Handle entirely missing variables
    for var in df_mis.columns[3:]:
        if df_mis[var].isna().all():
            # Replace with a default value, e.g., 0
            df_mis[var] = 0  # You can also use df_mis[var].mean() or another value
            '''

    # Step 2: Handle any remaining missing values after specific cases
    for var in df_mis.columns[3:]:
        df_mis[var].fillna(df_mis[var].median(), inplace=True)  # Replace remaining NaNs with median of the column

    return df_mis

# Apply the function to handle remaining missing values
df_filled_final = handle_remaining_missing_data(df_filled)

# Check if any missing values remain
print(df_filled_final.isna().sum())

company_name          0
company_code          0
date                  0
ask_price             0
asset_turnover        0
bid_price             0
cash_dividend         0
close_price           0
current_ratio         0
dividend_yield        0
dpps                  0
ebit                  0
ebitda                0
gross_pm              0
interest_cover1       0
interest_cover2       0
ltdce                 0
net_income            0
operating_pm          0
pe                    0
price_high            0
price_low             0
price_open            0
ptbv                  0
quick_ratio           0
roa                   0
roe                   0
roic                  0
shares_outstanding    0
tdce                  0
total_return          0
trading_volume_wa     0
turnover_value        0
turnover_volume       0
vwap                  0
dtype: int64


# Merge The Two Datasets #

In [23]:
# Importing S&P 500 Return Index and Volatility Index (VIX) from  2014-12-08 until 2024-12-02
sp500_index = pd.read_csv("weekly_sp500_vix.csv")
sp500_index.head()

,Date,SP500_Return (%),VIX_Close
0,2014-12-08,-3.519380,21.080000
1,2014-12-15,3.412022,16.490000
2,2014-12-22,0.875093,14.500000
3,2014-12-29,-1.463544,17.790001
4,2015-01-05,-0.650563,17.549999


In [24]:
sp500_index.tail()

,Date,SP500_Return (%),VIX_Close
517,2024-11-04,4.656128,14.940000
518,2024-11-11,-2.083547,16.139999
519,2024-11-18,1.681589,15.240000
520,2024-11-25,1.056064,13.510000
521,2024-12-02,0.290101,13.300000


In [25]:
# Move all dates in the "sp500_index" dataset 3 days backward to align with our main dataset
# Ensure the 'Date' column is in datetime format
sp500_index['Date'] = pd.to_datetime(sp500_index['Date'])

# Subtract 3 days from the 'Date' column
sp500_index['Date'] = sp500_index['Date'] - pd.Timedelta(days=3)

# View the modified DataFrame
sp500_index.head()

,Date,SP500_Return (%),VIX_Close
0,2014-12-05,-3.519380,21.080000
1,2014-12-12,3.412022,16.490000
2,2014-12-19,0.875093,14.500000
3,2014-12-26,-1.463544,17.790001
4,2015-01-02,-0.650563,17.549999


In [26]:
sp500_index.tail()

,Date,SP500_Return (%),VIX_Close
517,2024-11-01,4.656128,14.940000
518,2024-11-08,-2.083547,16.139999
519,2024-11-15,1.681589,15.240000
520,2024-11-22,1.056064,13.510000
521,2024-11-29,0.290101,13.300000


In [27]:
# Merge the two datasets

# Ensure the 'Date' columns in both datasets are datetime types
df_filled_final['date'] = pd.to_datetime(df_filled_final['date'])
sp500_index['Date'] = pd.to_datetime(sp500_index['Date'])

# Merge the datasets on the date column
merged_df = df_filled_final.merge(
    sp500_index.rename(columns={'Date': 'date'}),  # Rename for alignment
    on='date',
    how='left'  # Keep all rows in df_filled_final
)

# Display the merged dataset
merged_df.head()

,company_name,company_code,date,ask_price,asset_turnover,bid_price,cash_dividend,close_price,current_ratio,dividend_yield,...,roic,shares_outstanding,tdce,total_return,trading_volume_wa,turnover_value,turnover_volume,vwap,SP500_Return (%),VIX_Close
0,3M,US88579Y1010,2014-12-05,135.6365,1.05,135.6114,2216000.0,135.6699,1.96,2.11,...,22.79,635135.0,52.61,5622.58,1782344.0,849750.1,14274.8,135.7130,-3.519380,21.080000
1,3M,US88579Y1010,2014-12-12,131.4896,1.05,131.4812,2216000.0,131.3642,1.96,2.18,...,22.79,635135.0,52.61,5444.14,1782344.0,849750.1,14913.8,132.0684,3.412022,16.490000
2,3M,US88579Y1010,2014-12-19,138.3705,1.05,138.3621,2216000.0,138.3537,1.96,2.48,...,22.79,635135.0,52.61,5733.81,1782344.0,1006228.0,21761.2,138.1173,0.875093,14.500000
3,3M,US88579Y1010,2014-12-26,139.0477,1.05,139.0226,2216000.0,139.0059,1.96,2.47,...,22.79,635135.0,52.61,5760.84,1782344.0,394893.9,8217.5,139.2474,-1.463544,17.790001
4,3M,US88579Y1010,2015-01-02,137.1414,0.94,137.1163,2561000.0,137.1665,1.54,2.50,...,23.05,609330.0,92.61,5684.61,1821209.0,349564.0,7354.4,136.9276,-0.650563,17.549999


In [28]:
merged_df.tail()

,company_name,company_code,date,ask_price,asset_turnover,bid_price,cash_dividend,close_price,current_ratio,dividend_yield,...,roic,shares_outstanding,tdce,total_return,trading_volume_wa,turnover_value,turnover_volume,vwap,SP500_Return (%),VIX_Close
103205,ZEBRA TECHNOLOGIES 'A',US9892071054,2024-11-01,384.6499,0.67,384.4900,0.0,384.6399,1.05,0.0,...,8.01,51379.0,73.12,9484.27,535725.0,1593302.0,4194.4,384.1338,4.656128,14.940000
103206,ZEBRA TECHNOLOGIES 'A',US9892071054,2024-11-08,399.3000,0.67,399.0500,0.0,399.3101,1.05,0.0,...,8.01,51379.0,73.12,9846.00,535725.0,798172.1,2052.8,399.8765,-2.083547,16.139999
103207,ZEBRA TECHNOLOGIES 'A',US9892071054,2024-11-15,385.8899,0.67,385.7100,0.0,385.9099,1.05,0.0,...,8.01,51379.0,73.12,9515.59,535725.0,688635.1,1732.4,386.1726,1.681589,15.240000
103208,ZEBRA TECHNOLOGIES 'A',US9892071054,2024-11-22,397.1899,0.67,396.9900,0.0,397.1899,1.05,0.0,...,8.01,51379.0,73.12,9793.72,535725.0,710837.3,1836.0,396.1980,1.056064,13.510000
103209,ZEBRA TECHNOLOGIES 'A',US9892071054,2024-11-29,406.9900,0.67,406.6201,0.0,407.0000,1.05,0.0,...,8.01,51379.0,73.12,10035.61,535725.0,492827.2,1223.5,406.8760,0.290101,13.300000


In [29]:
# Check again for missing values after the merge
print(merged_df.isna().sum())

company_name          0
company_code          0
date                  0
ask_price             0
asset_turnover        0
bid_price             0
cash_dividend         0
close_price           0
current_ratio         0
dividend_yield        0
dpps                  0
ebit                  0
ebitda                0
gross_pm              0
interest_cover1       0
interest_cover2       0
ltdce                 0
net_income            0
operating_pm          0
pe                    0
price_high            0
price_low             0
price_open            0
ptbv                  0
quick_ratio           0
roa                   0
roe                   0
roic                  0
shares_outstanding    0
tdce                  0
total_return          0
trading_volume_wa     0
turnover_value        0
turnover_volume       0
vwap                  0
SP500_Return (%)      0
VIX_Close             0
dtype: int64


In [31]:
# Descriptive Analysis
# (to be done)

# Data Transformation, Standardization and Split #

In [30]:
# Check data types of all columns
print(merged_df.dtypes)

company_name                  object
company_code                  object
date                  datetime64[ns]
ask_price                    float64
asset_turnover               float64
bid_price                    float64
cash_dividend                float64
close_price                  float64
current_ratio                float64
dividend_yield               float64
dpps                         float64
ebit                         float64
ebitda                       float64
gross_pm                     float64
interest_cover1              float64
interest_cover2              float64
ltdce                        float64
net_income                   float64
operating_pm                 float64
pe                           float64
price_high                   float64
price_low                    float64
price_open                   float64
ptbv                         float64
quick_ratio                  float64
roa                          float64
roe                          float64
r

In [31]:
# Create a new column "weekly_return" based on close price, "return_close" will be our target feature

# Ensure the dataset is sorted by company and date
df_sorted = merged_df.sort_values(by=['company_name', 'date']).copy()

# Calculate weekly stock returns for each company
df_sorted['weekly_return'] = df_sorted.groupby('company_name')['close_price'].pct_change()

# Drop rows with NaN in 'return_close'
df_return = df_sorted.dropna(subset=['weekly_return'])

# Display the first few rows to verify
df_return.head()

,company_name,company_code,date,ask_price,asset_turnover,bid_price,cash_dividend,close_price,current_ratio,dividend_yield,...,shares_outstanding,tdce,total_return,trading_volume_wa,turnover_value,turnover_volume,vwap,SP500_Return (%),VIX_Close,weekly_return
1,3M,US88579Y1010,2014-12-12,131.4896,1.05,131.4812,2216000.0,131.3642,1.96,2.18,...,635135.0,52.61,5444.14,1782344.0,849750.1,14913.8,132.0684,3.412022,16.490000,-0.031737
2,3M,US88579Y1010,2014-12-19,138.3705,1.05,138.3621,2216000.0,138.3537,1.96,2.48,...,635135.0,52.61,5733.81,1782344.0,1006228.0,21761.2,138.1173,0.875093,14.500000,0.053207
3,3M,US88579Y1010,2014-12-26,139.0477,1.05,139.0226,2216000.0,139.0059,1.96,2.47,...,635135.0,52.61,5760.84,1782344.0,394893.9,8217.5,139.2474,-1.463544,17.790001,0.004714
4,3M,US88579Y1010,2015-01-02,137.1414,0.94,137.1163,2561000.0,137.1665,1.54,2.50,...,609330.0,92.61,5684.61,1821209.0,349564.0,7354.4,136.9276,-0.650563,17.549999,-0.013233
5,3M,US88579Y1010,2015-01-09,135.2352,0.94,135.2018,2561000.0,135.1265,1.54,2.54,...,609330.0,92.61,5600.06,1821209.0,703335.1,18945.1,135.4806,-1.241681,20.950001,-0.014872


In [32]:
df_return.tail()

,company_name,company_code,date,ask_price,asset_turnover,bid_price,cash_dividend,close_price,current_ratio,dividend_yield,...,shares_outstanding,tdce,total_return,trading_volume_wa,turnover_value,turnover_volume,vwap,SP500_Return (%),VIX_Close,weekly_return
103205,ZEBRA TECHNOLOGIES 'A',US9892071054,2024-11-01,384.6499,0.67,384.4900,0.0,384.6399,1.05,0.0,...,51379.0,73.12,9484.27,535725.0,1593302.0,4194.4,384.1338,4.656128,14.940000,0.068177
103206,ZEBRA TECHNOLOGIES 'A',US9892071054,2024-11-08,399.3000,0.67,399.0500,0.0,399.3101,1.05,0.0,...,51379.0,73.12,9846.00,535725.0,798172.1,2052.8,399.8765,-2.083547,16.139999,0.038140
103207,ZEBRA TECHNOLOGIES 'A',US9892071054,2024-11-15,385.8899,0.67,385.7100,0.0,385.9099,1.05,0.0,...,51379.0,73.12,9515.59,535725.0,688635.1,1732.4,386.1726,1.681589,15.240000,-0.033558
103208,ZEBRA TECHNOLOGIES 'A',US9892071054,2024-11-22,397.1899,0.67,396.9900,0.0,397.1899,1.05,0.0,...,51379.0,73.12,9793.72,535725.0,710837.3,1836.0,396.1980,1.056064,13.510000,0.029230
103209,ZEBRA TECHNOLOGIES 'A',US9892071054,2024-11-29,406.9900,0.67,406.6201,0.0,407.0000,1.05,0.0,...,51379.0,73.12,10035.61,535725.0,492827.2,1223.5,406.8760,0.290101,13.300000,0.024699


In [33]:
# Create a new column "label" based on 'weekly_return', assign "1" if next week's return is above the median, and assign "0" otherwise.

# Step 1: Shift weekly returns to get next week's performance
df_label = df_return.copy()  # Ensure df_return is a copy to avoid warnings
df_label['next_week_return'] = df_label.groupby('company_name')['weekly_return'].shift(-1)

# Step 2: Calculate the weekly median return for all companies
df_label['weekly_median_return'] = df_label.groupby('date')['next_week_return'].transform('median')

# Step 3: Assign labels based on next week's performance
# If next_week_return > weekly_median_return, assign 1; otherwise, assign 0
df_label['label'] = (df_label['next_week_return'] > df_label['weekly_median_return']).astype(int)

# Drop rows where next_week_return or weekly_median_return is NaN (e.g., the last week for each company)
df_label = df_label.dropna(subset=['next_week_return', 'weekly_median_return'])

# Display the first few rows to verify
df_label[['company_name', 'date', 'weekly_return', 'next_week_return', 'weekly_median_return', 'label']].head()

,company_name,date,weekly_return,next_week_return,weekly_median_return,label
1,3M,2014-12-12,-0.031737,0.053207,0.029329,1
2,3M,2014-12-19,0.053207,0.004714,0.012400,0
3,3M,2014-12-26,0.004714,-0.013233,-0.013382,1
4,3M,2015-01-02,-0.013233,-0.014872,-0.008866,0
5,3M,2015-01-09,-0.014872,0.002351,-0.012969,1


In [34]:
# Drop the 'weekly_median_return' column after labeling
df_label = df_label.drop(columns=['weekly_median_return'])

df_label

,company_name,company_code,date,ask_price,asset_turnover,bid_price,cash_dividend,close_price,current_ratio,dividend_yield,...,total_return,trading_volume_wa,turnover_value,turnover_volume,vwap,SP500_Return (%),VIX_Close,weekly_return,next_week_return,label
1,3M,US88579Y1010,2014-12-12,131.4896,1.05,131.4812,2216000.0,131.3642,1.96,2.18,...,5444.14,1782344.0,849750.1,14913.8,132.0684,3.412022,16.490000,-0.031737,0.053207,1
2,3M,US88579Y1010,2014-12-19,138.3705,1.05,138.3621,2216000.0,138.3537,1.96,2.48,...,5733.81,1782344.0,1006228.0,21761.2,138.1173,0.875093,14.500000,0.053207,0.004714,0
3,3M,US88579Y1010,2014-12-26,139.0477,1.05,139.0226,2216000.0,139.0059,1.96,2.47,...,5760.84,1782344.0,394893.9,8217.5,139.2474,-1.463544,17.790001,0.004714,-0.013233,1
4,3M,US88579Y1010,2015-01-02,137.1414,0.94,137.1163,2561000.0,137.1665,1.54,2.50,...,5684.61,1821209.0,349564.0,7354.4,136.9276,-0.650563,17.549999,-0.013233,-0.014872,0
5,3M,US88579Y1010,2015-01-09,135.2352,0.94,135.2018,2561000.0,135.1265,1.54,2.54,...,5600.06,1821209.0,703335.1,18945.1,135.4806,-1.241681,20.950001,-0.014872,0.002351,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103204,ZEBRA TECHNOLOGIES 'A',US9892071054,2024-10-25,360.0901,0.67,359.9600,0.0,360.0901,1.05,0.00,...,8878.93,535725.0,550593.1,1503.0,362.2358,-1.365680,21.879999,-0.040911,0.068177,1
103205,ZEBRA TECHNOLOGIES 'A',US9892071054,2024-11-01,384.6499,0.67,384.4900,0.0,384.6399,1.05,0.00,...,9484.27,535725.0,1593302.0,4194.4,384.1338,4.656128,14.940000,0.068177,0.038140,0
103206,ZEBRA TECHNOLOGIES 'A',US9892071054,2024-11-08,399.3000,0.67,399.0500,0.0,399.3101,1.05,0.00,...,9846.00,535725.0,798172.1,2052.8,399.8765,-2.083547,16.139999,0.038140,-0.033558,0
103207,ZEBRA TECHNOLOGIES 'A',US9892071054,2024-11-15,385.8899,0.67,385.7100,0.0,385.9099,1.05,0.00,...,9515.59,535725.0,688635.1,1732.4,386.1726,1.681589,15.240000,-0.033558,0.029230,1


In [35]:
# standardize data

stock = df_label.copy() # Make a copy of df_label

X_cols = ["close_price", "ask_price", "bid_price", "price_high", "price_low", "price_open",
         "turnover_value", "turnover_volume", "vwap", "trading_volume_wa",
         "total_return", "pe", "ptbv", "dividend_yield", "operating_pm", "gross_pm",
         "net_income", "roe", "roic", "roa", "ebit", "ebitda", "dpps",
         "current_ratio", "quick_ratio", "asset_turnover",
         "tdce", "ltdce", "interest_cover1", "interest_cover2", "cash_dividend", "shares_outstanding",
         "SP500_Return (%)", "VIX_Close", "weekly_return"]

y_col = ['label']

X = stock[X_cols]
y = stock[y_col]

scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

stock_scaled = stock.copy()

for i in range(len(X_cols)):
    stock_scaled[X_cols[i]] = X_scaled[:,i]

stock_scaled

,company_name,company_code,date,ask_price,asset_turnover,bid_price,cash_dividend,close_price,current_ratio,dividend_yield,...,total_return,trading_volume_wa,turnover_value,turnover_volume,vwap,SP500_Return (%),VIX_Close,weekly_return,next_week_return,label
1,3M,US88579Y1010,2014-12-12,-0.037403,0.701259,-0.037260,0.552046,-0.037671,0.167149,0.187528,...,-0.235380,-0.089593,-0.180834,-0.153202,-0.035605,1.356102,-0.220277,-0.772645,0.053207,1
2,3M,US88579Y1010,2014-12-19,-0.018894,0.701259,-0.018732,0.552046,-0.018864,0.167149,0.356054,...,-0.232564,-0.089593,-0.163759,-0.117334,-0.019323,0.271328,-0.501640,1.101534,0.004714,0
3,3M,US88579Y1010,2014-12-26,-0.017072,0.701259,-0.016953,0.552046,-0.017109,0.167149,0.350436,...,-0.232301,-0.089593,-0.230467,-0.188279,-0.016281,-0.728657,-0.036471,0.031594,-0.013233,1
4,3M,US88579Y1010,2015-01-02,-0.022200,0.523145,-0.022087,0.718940,-0.022058,-0.101185,0.367289,...,-0.233042,-0.084981,-0.235413,-0.192800,-0.022525,-0.381032,-0.070405,-0.364375,-0.014872,0
5,3M,US88579Y1010,2015-01-09,-0.027327,0.523145,-0.027242,0.718940,-0.027548,-0.101185,0.389759,...,-0.233864,-0.084981,-0.196810,-0.132085,-0.026420,-0.633790,0.410316,-0.400558,0.002351,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103204,ZEBRA TECHNOLOGIES 'A',US9892071054,2024-10-25,0.577528,0.085954,0.577972,-0.519941,0.577791,-0.414241,-1.037092,...,-0.201990,-0.237510,-0.213477,-0.223450,0.583935,-0.686811,0.541807,-0.975059,0.068177,1
103205,ZEBRA TECHNOLOGIES 'A',US9892071054,2024-11-01,0.643593,0.085954,0.644025,-0.519941,0.643850,-0.414241,-1.037092,...,-0.196105,-0.237510,-0.099699,-0.209352,0.642878,1.888073,-0.439429,1.431824,0.038140,0
103206,ZEBRA TECHNOLOGIES 'A',US9892071054,2024-11-08,0.683001,0.085954,0.683231,-0.519941,0.683325,-0.414241,-1.037092,...,-0.192588,-0.237510,-0.186462,-0.220570,0.685252,-0.993766,-0.269763,0.769100,-0.033558,0
103207,ZEBRA TECHNOLOGIES 'A',US9892071054,2024-11-15,0.646929,0.085954,0.647310,-0.519941,0.647267,-0.414241,-1.037092,...,-0.195800,-0.237510,-0.198414,-0.222249,0.648366,0.616181,-0.397012,-0.812840,0.029230,1


In [36]:
# Sort DataFrame first by date and the by company_name
stock_scaled_sorted = stock_scaled.sort_values(by=['date', 'company_name'])
stock_scaled_sorted = stock_scaled_sorted.reset_index(drop=True)

stock_scaled_sorted

,company_name,company_code,date,ask_price,asset_turnover,bid_price,cash_dividend,close_price,current_ratio,dividend_yield,...,total_return,trading_volume_wa,turnover_value,turnover_volume,vwap,SP500_Return (%),VIX_Close,weekly_return,next_week_return,label
0,3M,US88579Y1010,2014-12-12,-0.037403,0.701259,-0.037260,0.552046,-0.037671,0.167149,0.187528,...,-0.235380,-0.089593,-0.180834,-0.153202,-0.035605,1.356102,-0.220277,-0.772645,0.053207,1
1,AFLAC,US0010551028,2014-12-12,-0.312694,-0.691274,-0.312825,-0.203569,-0.312739,0.005057,0.468404,...,1.022079,-0.231122,-0.180834,-0.100681,-0.312083,1.356102,-0.220277,-0.528447,0.049760,1
2,ALLSTATE ORD SHS,US0200021014,2014-12-12,-0.209722,-0.464583,-0.209760,-0.247106,-0.209734,0.005057,-0.104583,...,-0.280835,-0.217311,-0.180834,-0.158125,-0.208070,1.356102,-0.220277,-0.477196,0.036636,1
3,ALPHABET 'A',US02079K3059,2014-12-12,-0.320932,-0.189314,-0.321091,-0.519941,-0.320985,1.981598,-1.037092,...,-0.278208,0.552743,-0.180834,1.042778,-0.320299,1.356102,-0.220277,-0.346917,-0.002819,0
4,ALPHABET 'C',US02079K1079,2014-12-12,-0.321526,-0.189314,-0.321665,-0.519941,-0.321560,1.981598,-1.037092,...,-0.287419,0.552743,-0.180834,0.768274,-0.320971,1.356102,-0.220277,-0.349654,-0.004454,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102805,WESTERN DIGITAL,US9581021055,2024-11-22,-0.212385,-0.124545,-0.212453,-0.519941,-0.212398,-0.241741,-1.037092,...,-0.276628,-0.197066,-0.134745,-0.128948,-0.212511,0.348710,-0.641614,1.202928,0.098751,1
102806,WILLIAMS,US9694571004,2024-11-22,-0.230704,-0.691274,-0.230791,0.534148,-0.230642,-0.593130,0.754898,...,-0.168749,-0.174656,-0.054421,-0.052238,-0.230244,0.348710,-0.641614,1.137094,-0.018944,0
102807,WILLIS TOWERS WATSON,IE00BDB6Q211,2024-11-22,0.454811,-0.464583,0.455022,-0.349661,0.454846,-0.407852,-0.407929,...,-0.276990,-0.224697,-0.167518,-0.214970,0.457243,0.348710,-0.641614,0.051083,0.024173,1
102808,WW GRAINGER,US3848021040,2024-11-22,2.852335,2.271907,2.855434,-0.330311,2.855738,0.754928,-0.655100,...,0.306480,-0.193096,-0.165304,-0.226936,2.846998,0.348710,-0.641614,0.457866,-0.001086,0


In [39]:
# Split the training and test dataset

# Define the target column
target_col = stock_scaled_sorted['label']

# Define the columns to exclude
exclude_cols = ['company_name', 'company_code', 'date', 'next_week_return', 'label']

# Get the input columns by excluding the specified columns
input_cols = stock_scaled_sorted.columns.difference(exclude_cols)

# Ensure all column names are strings
stock_scaled_sorted.columns = stock_scaled_sorted.columns.astype(str)

# Split the data into inputs (features) and target
X = stock_scaled_sorted[input_cols]
y = stock_scaled_sorted['label']

# Split the data into 80% training and 20% testing
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Stratified split
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_idx, test_idx in splitter.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

# Verifying the shapes
print("Training set:", X_train.shape, y_train.shape)
print("Testing set:", X_test.shape, y_test.shape)

Training set: (82248, 35) (82248,)
Testing set: (20562, 35) (20562,)


# Random Forest Classification Method to Rank the Stocks #

In [40]:
# Step 1: Train the Random Forest Classifier with default parameters

# Initialize Random Forest Classifier
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate accuracy on the test set
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

Model Accuracy: 0.55


In [41]:
# Get feature importance
importances = rf_model.feature_importances_
feature_names = X_train.columns
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})

# Sort by importance
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
# feature_importance_df.to_csv("feature_importance.csv")

feature_importance_df

,Feature,Importance
0,SP500_Return (%),0.081017
34,weekly_return,0.067779
1,VIX_Close,0.065040
31,turnover_value,0.058016
32,turnover_volume,0.055937
18,pe,0.050743
29,total_return,0.047207
22,ptbv,0.046118
21,price_open,0.041649
20,price_low,0.040903


In [ ]:
# Fine-tune hyperparameters (This process took 4 hours 28 minutes!!!)
'''
# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    cv=3,
    scoring='accuracy',
    verbose=2,
    n_jobs=-1
)

# Fit the model
grid_search.fit(X_train, y_train)

# Use the best model
best_rf_model = grid_search.best_estimator_
print(f"Best Parameters: {grid_search.best_params_}")
'''

Best Parameters got from the above fine-tuning process: {'max_depth': 20, 'max_features': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 500}

In [42]:
# Initialize Random Forest Classifier with best parameters (this block of code takes 15 minutes to run)
rf_model_best = RandomForestClassifier(random_state=42, n_estimators=500, max_depth=20,
                               min_samples_split=5, min_samples_leaf=2, max_features=None)  # with best parameters
rf_model_best.fit(X_train, y_train)

# Evaluate accuracy on the test set
y_pred = rf_model_best.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy after fine-tuning: {accuracy:.2f}")

Model Accuracy after fine-tuning: 0.58


We can see the accuracy after fine-tuning is still very low, we need to come up with a solution.